# Place an order

In [1]:
import requests
# sandbox api base
api_base = 'https://api-public.sandbox.gdax.com'
response = requests.get(api_base + '/products')
print(response.json())

[{'base_max_size': '10000', 'id': 'BTC-USD', 'quote_increment': '0.01', 'margin_enabled': True, 'display_name': 'BTC/USD', 'quote_currency': 'USD', 'status_message': None, 'base_currency': 'BTC', 'base_min_size': '0.01', 'status': 'online'}, {'base_max_size': '10000', 'id': 'BTC-GBP', 'quote_increment': '0.01', 'margin_enabled': False, 'display_name': 'BTC/GBP', 'quote_currency': 'GBP', 'status_message': None, 'base_currency': 'BTC', 'base_min_size': '0.01', 'status': 'online'}, {'base_max_size': '10000', 'id': 'BTC-EUR', 'quote_increment': '0.01', 'margin_enabled': False, 'display_name': 'BTC/EUR', 'quote_currency': 'EUR', 'status_message': None, 'base_currency': 'BTC', 'base_min_size': '0.01', 'status': 'online'}, {'base_max_size': '1000000', 'id': 'ETH-BTC', 'quote_increment': '0.00001', 'margin_enabled': False, 'display_name': 'ETH/BTC', 'quote_currency': 'BTC', 'status_message': None, 'base_currency': 'ETH', 'base_min_size': '0.01', 'status': 'online'}, {'base_max_size': '1000000'

### Notes

- We are going to keep track of the **product_id** field in order to palce an order later.
- Covert code to function

In [2]:
def products():
    response = requests.get(api_base + '/products')
    # check for invalid api response
    if response.status_code is not 200:
        raise Exception('Invalid GDAX Status Code: %d' % response.status_code)
    return response.json()

In [3]:
products()

[{'base_currency': 'BTC',
  'base_max_size': '10000',
  'base_min_size': '0.01',
  'display_name': 'BTC/USD',
  'id': 'BTC-USD',
  'margin_enabled': True,
  'quote_currency': 'USD',
  'quote_increment': '0.01',
  'status': 'online',
  'status_message': None},
 {'base_currency': 'BTC',
  'base_max_size': '10000',
  'base_min_size': '0.01',
  'display_name': 'BTC/GBP',
  'id': 'BTC-GBP',
  'margin_enabled': False,
  'quote_currency': 'GBP',
  'quote_increment': '0.01',
  'status': 'online',
  'status_message': None},
 {'base_currency': 'BTC',
  'base_max_size': '10000',
  'base_min_size': '0.01',
  'display_name': 'BTC/EUR',
  'id': 'BTC-EUR',
  'margin_enabled': False,
  'quote_currency': 'EUR',
  'quote_increment': '0.01',
  'status': 'online',
  'status_message': None},
 {'base_currency': 'ETH',
  'base_max_size': '1000000',
  'base_min_size': '0.01',
  'display_name': 'ETH/BTC',
  'id': 'ETH-BTC',
  'margin_enabled': False,
  'quote_currency': 'BTC',
  'quote_increment': '0.00001',
 

# Getting Authentication Working

- The products endpoint was public, so there wasn’t any authentication required. 
- But many other actions (including placing an order) do require authentication.

#### There are 4 auth headers that we need to pass with every request:

- **CB-ACCESS-KEY**: The API key that we generated earlier.
- **CB-ACCESS-PASSPHRASE**: The passphrase that was shown when creating the API key.
- **CB-ACCESS-TIMESTAMP**: A timestamp for the request (the number of seconds since the Unix Epoch in UTC - coordinated universal time). We can generate this using time.time().
- **CB-ACCESS-SIGN**: [The base64-encoded message signature](https://crypto.stackexchange.com/questions/35060/how-is-the-base64-encoded-signature-in-craig-wrights-proof-constructed).

## How do we sign a message?

According to the API documentation, the message signature header is generated in four steps:

- Combine the timestamp, request method, request path, and body into a string. 
- base64-encode the combined string.
- Generate a [HMAC key](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=3&cad=rja&uact=8&ved=0ahUKEwjqiuX3kvfXAhUM3mMKHfnLD9QQFggvMAI&url=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FHash-based_message_authentication_code&usg=AOvVaw1tSM2Ftz2egrVTpmA26nVF) by base64-decoding the API secret key.
- Use the HMAC key to create a sha256 HMAC with the encoded combined string.

In [17]:
import base64, hashlib, hmac, time
from requests.auth import AuthBase

class GDAXRequestAuth(AuthBase):
    def __init__(self, api_key, secret_key, passphrase):
        self.api_key = api_key
        self.secret_key = secret_key
        self.passphrase = passphrase
    
    def __call__(self, request):
        timestamp = str(time.time())
        message = timestamp + request.method + request.path_url + (request.body or '')
        hmac_key = base64.b64decode(self.secret_key)
        signature = hmac.new(hmac_key, message.encode('utf-8'), hashlib.sha256)
        signature_b64 = base64.b64encode(signature.digest())
        request.headers.update({
            'CB-ACCESS-SIGN': signature_b64,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-ACCESS-PASSPHRASE': self.passphrase,
            'Content-Type': 'application/json'
        })
        return request

In [18]:
auth = GDAXRequestAuth(api_key, api_secret, passphrase)
response = requests.post(order_url, data=data, auth=auth)

NameError: name 'api_key' is not defined

In [20]:
import json

api_base = 'https://api-public.sandbox.gdax.com'
api_key = 'a5fb0d9876445d9c38eed687eab5f2d9'
api_secret = 'wxn1vztWzQBGfMFZdKRr93UJAf48AHw0fPRD07Z7r38IgoJ/AyC9aS7qeXuVt2OjKlH9KTF9o8rAfrAS6U/itQ=='
passphrase = 'sf4omjcl6d'

# use our request auth object
auth = GDAXRequestAuth(api_key, api_secret, passphrase)
order_url = api_base + '/orders'
order_data = {
    'type': 'market',
    'side': 'buy',
    'product_id': 'BTC-USD',
    'size': '0.01'
}
response = requests.post(order_url, data=json.dumps(order_data), auth=auth)
print(response.json())

{'executed_value': '0.0000000000000000', 'created_at': '2017-12-07T05:50:11.920129Z', 'size': '0.01000000', 'post_only': False, 'stp': 'dc', 'type': 'market', 'id': '5f15ea32-3d8b-4c5a-b2e3-b42c8f312bd1', 'status': 'pending', 'side': 'buy', 'product_id': 'BTC-USD', 'settled': False, 'funds': '299.2559625900000000', 'fill_fees': '0.0000000000000000', 'filled_size': '0.00000000'}


- However, if you look closely at our results, you’ll notice that the status field of the order is set to pending.
- Before we can be sure that our order went through, we’ll need to use the order status endpoint (/order/<order-id>). 
- The order-id value is taken from the id field of the previous response.

In [22]:
order_id = '5f15ea32-3d8b-4c5a-b2e3-b42c8f312bd1'
order_url = api_base + '/orders/' + order_id
response = requests.get(order_url, auth=auth)
print(response.json())

{'executed_value': '299.2500000000000000', 'created_at': '2017-12-07T05:50:11.920129Z', 'side': 'buy', 'size': '0.01000000', 'post_only': False, 'stp': 'dc', 'type': 'market', 'id': '5f15ea32-3d8b-4c5a-b2e3-b42c8f312bd1', 'status': 'done', 'done_reason': 'filled', 'product_id': 'BTC-USD', 'settled': True, 'done_at': '2017-12-07T05:50:11.925Z', 'funds': '299.2559625900000000', 'fill_fees': '0.7481250000000000', 'filled_size': '0.00033250'}


-  turn these endpoint calls into buy_market and order_status functions for easier re-use.

In [35]:
def buy_market(product_id, size):
    auth = GDAXRequestAuth(api_key, api_secret, passphrase)
    order_data = {
        'type': 'market',
        'side': 'buy',
        'product_id': product_id,
        'size': size
    }
    response = requests.post(api_base + '/orders', data=json.dumps(order_data), auth=auth)
    if response.status_code is not 200:
        raise Exception('Invalid GDAX Status Code: %d' % response.status_code)
    return response.json()

def sell_market(product_id, size):
    auth = GDAXRequestAuth(api_key, api_secret, passphrase)
    order_data = {
        'type': 'market',
        'side': 'sell',
        'product_id': product_id,
        'size': size
    }
    response = requests.post(api_base + '/orders', data=json.dumps(order_data), auth=auth)
    if response.status_code is not 200:
        raise Exception('Invalid GDAX Status Code: %d' % response.status_code)
    return response.json()

def order_status(order_id):
    order_url = api_base + '/orders/' + order_id
    response = requests.get(order_url, auth=auth)
    if response.status_code is not 200:
        raise Exception('Invalid GDAX Status Code: %d' % response.status_code)
    return response.json()

In [53]:
product_id = 'BTC-USD'
buy_market(product_id,'0.01')

{'created_at': '2017-12-07T06:17:53.109956Z',
 'executed_value': '0.0000000000000000',
 'fill_fees': '0.0000000000000000',
 'filled_size': '0.00000000',
 'funds': '39900.2506857800000000',
 'id': '7ee10baf-f0ed-4897-95f2-ccfbbd354288',
 'post_only': False,
 'product_id': 'BTC-USD',
 'settled': False,
 'side': 'buy',
 'size': '0.01000000',
 'status': 'pending',
 'stp': 'dc',
 'type': 'market'}

In [54]:
order_status('12d739bc-5467-4fd5-81f6-b84f287afcb3')

{'created_at': '2017-12-07T06:10:21.824301Z',
 'done_at': '2017-12-07T06:10:21.834Z',
 'done_reason': 'filled',
 'executed_value': '2992.5180000000000000',
 'fill_fees': '7.4812950000000000',
 'filled_size': '0.00332502',
 'funds': '2992.5246658300000000',
 'id': '12d739bc-5467-4fd5-81f6-b84f287afcb3',
 'post_only': False,
 'product_id': 'BTC-USD',
 'settled': True,
 'side': 'buy',
 'size': '0.01000000',
 'status': 'done',
 'stp': 'dc',
 'type': 'market'}

In [55]:
sell_market(product_id, '0.01')

{'created_at': '2017-12-07T06:17:58.507264Z',
 'executed_value': '0.0000000000000000',
 'fill_fees': '0.0000000000000000',
 'filled_size': '0.00000000',
 'id': 'cb738d59-73cf-42df-b54d-7c85e389a185',
 'post_only': False,
 'product_id': 'BTC-USD',
 'settled': False,
 'side': 'sell',
 'size': '0.01000000',
 'status': 'pending',
 'stp': 'dc',
 'type': 'market'}

### API call

- Market orders are great, but in order to make any headway into more sophisticated trading we’re going to need to do limit orders. 
- A limit order allows you to set the price at which you’re willing to buy (or sell) a coin, and it will execute the trade at that price or better. 
- In our case, let’s see we want to buy a single Ether at a price of $250$ or better.

- For limit orders, there are a few new parameters that we need to send: 
    - price (400) 
    - and size (1) are the only two that are required,
- but there are a few optional ones as well. 
    - time_in_force defines how long we want the limit order to last for
    - , and there are a **few options to choose from (see below)**. 
        - cancel_after is used in conjunction with time_in_force, 
        - and will set a time limit to cancel orders after a certain amount of time. 
        
- Lastly, the post_only flag specifies that the order should only make liquidity, and will otherwise result in a rejected order.

### Time in force policies:

- GTC: Good till canceled. Orders remain upon until they’re manually canceled.
- GTT Good till time. GTT orders can set a cancel_after parameter that will cancel the order after a certain amount of time (e.g. minute, hour, day).
- IOC: Immediate or cancel. If the entire order size cannot be filled, the order is partially filled and the rest is canceled.
- FOK: Fill or kill. If the entire order size cannot be filled, the order is rejected.

In [61]:
def buy_limit(product_id, price, size, time_in_force='GTC', cancel_after=None, post_only=None):
    auth = GDAXRequestAuth(api_key, api_secret, passphrase)
    order_data = {
        'type': 'limit',
        'side': 'buy',
        'product_id': product_id,
        'price': price,
        'size': size,
        'time_in_force': time_in_force
    }
    if 'time_in_force' is 'GTT':
        order_data['cancel_after'] = cancel_after 
    if 'time_in_force' not in ['IOC', 'FOK']:
        order_data['post_only'] = post_only
    response = requests.post(api_base + '/orders', data=json.dumps(order_data), auth=auth)
    if response.status_code is not 200:
        raise Exception('Invalid GDAX Status Code: %d' % response.status_code)
    return response.json()

def sell_limit(product_id, price, size, time_in_force='GTC', cancel_after=None, post_only=None):
    auth = GDAXRequestAuth(api_key, api_secret, passphrase)
    order_data = {
        'type': 'limit',
        'side': 'sell',
        'product_id': product_id,
        'price': price,
        'size': size,
        'time_in_force': time_in_force
    }
    if 'time_in_force' is 'GTT':
        order_data['cancel_after'] = cancel_after 
    if 'time_in_force' not in ['IOC', 'FOK']:
        order_data['post_only'] = post_only
    response = requests.post(api_base + '/orders', data=json.dumps(order_data), auth=auth)
    if response.status_code is not 200:
        raise Exception('Invalid GDAX Status Code: %d' % response.status_code)
    return response.json()

In [83]:
price = '1111'
size = '0.01'
buy_limit(product_id, price, size, time_in_force='GTT', cancel_after=None, post_only=None)

Exception: Invalid GDAX Status Code: 400

In [73]:
price = '111'
size = '0.01'
sell_limit(product_id, price, size, time_in_force='GTC', cancel_after=None, post_only=False)

{'created_at': '2017-12-07T06:58:08.236135Z',
 'executed_value': '0.0000000000000000',
 'fill_fees': '0.0000000000000000',
 'filled_size': '0.00000000',
 'id': '33ce7ec9-3307-41a6-abbb-56b982768191',
 'post_only': False,
 'price': '111.00000000',
 'product_id': 'BTC-USD',
 'settled': False,
 'side': 'sell',
 'size': '0.01000000',
 'status': 'pending',
 'stp': 'dc',
 'time_in_force': 'GTC',
 'type': 'limit'}

# Fees

### acknowledge the fees structure of GDAX. 
- Rather than flat fees on buying and selling, GDAX distinguishes between “Makers” and “Takers,” 
    - which are named for transactions that provide liquidity to a market or remove liquidity from a market.
    
#### Here’s an example that they describe.
- There is an existing SELL order for 5 BTC at 100 USD on the order book. 
- You enter a BUY order for 7 BTC at 100 USD. 
- 5 BTC of your BUY order are immediately matched and you are charged the taker fee because you are taking liquidity from the order book. 
- The remaining 2 BTC of your order are now sitting on the BID side of the order book. 
- A SELL order for 2 BTC at 100 USD arrives and matches against your 2 BTC BUY order. In this case you provided liquidity and are not charged any fees.

- The maker fee is always 0%, which is great! 
- The taker fee starts at 0.25% (0.3% for ETH) and decreases according to your volume. 
- However, the in order to receive any discounts, you have to make up a pretty significant percentage of GDAX trading volume (at least 1% of the last 30-days worth of volume). 
- Unless you’re trading thousands of Bitcoin or Ether, you’re unlikely to wind up with fee discounts.

## There are many more API endpoints to cover, 
#### and way more complex things that you can do with the API. For example, you can:

- Automate deposits and withdrawals to and from bank accounts
- Enable margin trading to trade with more money (or short-sell)
- Scan the order book for arbitrage opportunities
- Use web sockets to get the realtime market data